In [1]:
from tqdm.contrib.concurrent import process_map

from notebooks.ford.asbuilt import AsBuiltData
from notebooks.ford.decode import search, print_breakdown


# openpilot current min model year is 2020 (excluding Focus Mk4/Kuga)
df_nhtsa = await search(
  min_model_year=2020,
  include_openpilot=True,
  skip_missing_asbuilt=True,
)

# pre-load asbuilt
# process_map(AsBuiltData.from_vin, df_nhtsa['VIN'].unique(), desc='Loading AsBuilt Data', chunksize=100)

print()
print_breakdown(df_nhtsa, include_model_year=False)

Loaded 36765 VINs (filter_comment=None, include_openpilot=True, skipped=163, missing_asbuilt=0)



Model
Aviator            1107
Bronco              997
Bronco Sport       1500
Continental          15
Corsair             986
E-Transit           157
Ecosport            237
Edge               1875
Escape             2613
Expedition          508
Expedition MAX      447
Explorer           2668
F-150              4848
F-150 Lightning     385
F-250               935
F-350               719
F-450               156
F-550                 1
Fusion              465
GT                    3
MKZ                  74
Maverick            994
Mustang             723
Mustang Mach-E      839
Nautilus            890
Navigator           292
Navigator L         217
Ranger              318
Transit             672
Transit Connect     204
dtype: int64


In [17]:
from notebooks.ford.platforms import find_openpilot_platform

df_cars = df_nhtsa[['Make', 'Model', 'ModelYear', 'Trim', 'Series', 'ElectrificationLevel']].drop_duplicates(ignore_index=True)

df_cars['CarName'] = df_cars['Make'] + ' ' + df_cars['Model'] + ' ' + df_cars['ModelYear'].astype(str)
df_cars['CarInfoPlatform'] = df_cars['CarName'].apply(find_openpilot_platform)
df_cars = df_cars[['CarInfoPlatform', 'ModelYear', 'ElectrificationLevel', 'Series', 'Trim']].dropna()

df_cars.head(n=10)

,CarInfoPlatform,ModelYear,ElectrificationLevel,Series,Trim
0,FORD EXPLORER 6TH GEN,2021,ICE,Limited,
5,FORD BRONCO SPORT 1ST GEN,2022,ICE,BIG BEND,
10,FORD ESCAPE 4TH GEN,2021,ICE,SE,
13,FORD EXPLORER 6TH GEN,2021,ICE,Base,
18,FORD F-150 14TH GEN,2023,ICE,,SuperCrew
20,FORD MUSTANG MACH-E 1ST GEN,2023,BEV,Premium,
21,FORD F-150 14TH GEN,2022,ICE,,SuperCrew
23,FORD ESCAPE 4TH GEN,2020,ICE,SE,
24,FORD F-150 14TH GEN,2021,ICE,F-Series,SuperCrew
28,FORD ESCAPE 4TH GEN,2021,ICE,Titanium,


In [23]:
import pandas as pd

pd.set_option('display.max_colwidth', None)

df = df_cars.groupby(['CarInfoPlatform', 'ElectrificationLevel'], as_index=False).agg(lambda x: ', '.join(map(lambda x: str(x), sorted(set(x)))))

print(df.to_markdown(index=False))
df

| CarInfoPlatform              | ElectrificationLevel   | ModelYear              | Series                                                                                            | Trim                                                                |
|:-----------------------------|:-----------------------|:-----------------------|:--------------------------------------------------------------------------------------------------|:--------------------------------------------------------------------|
| FORD BRONCO SPORT 1ST GEN    | ICE                    | 2021, 2022             | BADLANDS, BASE, BIG BEND, FIRST EDITION, OUTER BANKS                                              |                                                                     |
| FORD ESCAPE 4TH GEN          | FHEV                   | 2020, 2021, 2022       | SE, SEL, Titanium                                                                                 |                                                           

,CarInfoPlatform,ElectrificationLevel,ModelYear,Series,Trim
0,FORD BRONCO SPORT 1ST GEN,ICE,"2021, 2022","BADLANDS, BASE, BIG BEND, FIRST EDITION, OUTER BANKS",
1,FORD ESCAPE 4TH GEN,FHEV,"2020, 2021, 2022","SE, SEL, Titanium",
2,FORD ESCAPE 4TH GEN,ICE,"2020, 2021, 2022","S, SE, SEL, Titanium",
3,FORD ESCAPE 4TH GEN,PHEV,"2020, 2021, 2022","SE, SEL, Titanium",
4,FORD EXPLORER 6TH GEN,HEV,"2020, 2021, 2022, 2023","Limited, Platinum",
5,FORD EXPLORER 6TH GEN,ICE,"2020, 2021, 2022, 2023","Base, Black Label, King Ranch, Limited, Platinum, Reserve, ST, ST-Line, Standard, Timberline, XLT",
6,FORD EXPLORER 6TH GEN,PHEV,"2020, 2021","Black Label Grand Touring, Blk Label Grand Touring, Grand Touring",
7,FORD F-150 14TH GEN,HEV,"2021, 2022, 2023",", F-Series",SuperCrew
8,FORD F-150 14TH GEN,ICE,"2021, 2022, 2023",", F-Series",", Regular Cab, SuperCab, SuperCrew, SuperCrew-Raptor, SuperCrew-SSV"
9,FORD F-150 LIGHTNING 1ST GEN,BEV,"2022, 2023",,SuperCrew
